In [1]:
import pyrebase
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import sklearn.model_selection
import mglearn

In [2]:
config = {
  "apiKey": "XXX",
  "authDomain": "XXX",
  "databaseURL": "XXX",
    "storageBucket": "XXX",
    "serviceAccount": "/Users/YuanShen1/Desktop/projects/Assembo/Unknown"
}

In [3]:
firebase = pyrebase.initialize_app(config)

In [4]:
db = firebase.database()

In [5]:
users = db.child("users").get()

In [6]:
all_users = db.child("users").get()
users = {}
for user in all_users.each():
    #print(user.key()) # Morty
    #print(user.val()) # {name": "Mortimer 'Morty' Smith"}
    if("intro" in user.val()):
        users[user.key()] = user.val()['intro']

In [7]:
user_ids = [] 
user_values = []
for key, value in users.items():
    user_ids.append(key)
    user_values.append(value)


In [8]:
stemmer = PorterStemmer()
word_tokenizer = WordPunctTokenizer()
processed_user_values = []
for value in user_values:
    sent_tokenizer = PunktSentenceTokenizer(value)
    new_sentences = []
    for sentence in sent_tokenizer.tokenize(value):
        stemmed_words = [stemmer.stem(word).lower() for word in word_tokenizer.tokenize(sentence)]
        filtered_word = []
        for word in stemmed_words:
            if len(word) > 1:
                filtered_word.append(word)
        new_sentences.append(" ".join(filtered_word))
    processed_user_values.append(".".join(new_sentences))

In [9]:
processed_user_values

['my address educ my name my number my student email bs comput scienc honor state univers cumul gpa 27 relev coursework fundament of comput scienc discret mathemat program in system fundament data structur experi to child develop llc juli 2015 present expect graduat may 2017 it consult address taught founder how to creat and maintain databas of client use microsoft access.work on applic to the new york state to be certifi as facil that help children with disabl.submit spreadsheet of certifi child develop therapist.success help founder kick start her compani and websit is under construct.eclips contract june 2014 present network administr address start out as an estim intern.complet train as well as contract with burberri metropolitan museum of art columbia univers and more for the compani.respons for monitor the compani server for malici activ.taught co worker how to implement googl busi and cloud comput.project octob 2013 as final project for class creat miner node simul which mine da

In [10]:
vect = CountVectorizer(max_features=10000, max_df=.15, stop_words="english")

In [11]:
X = vect.fit_transform(processed_user_values)

In [12]:
lda = LatentDirichletAllocation(n_topics=10, learning_method="batch", max_iter=25, random_state=0)

In [13]:
document_topics = lda.fit_transform(X)

In [15]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]

In [16]:
feature_names = np.array(vect.get_feature_names())

In [28]:
mglearn.tools.print_topics(topics=range(6), feature_names=feature_names,
                               sorting=sorting, topics_per_chunk=10, n_words=30)

topic 0       topic 1       topic 2       topic 3       topic 4       topic 5       
--------      --------      --------      --------      --------      --------      
jkl           legaci        seo           task          def           jkl           
bioinformat   ri            oh            profession    bsse          bioinformat   
print         2002          interact      georgina      bioinformat   print         
premium       intel         best          19xx          print         premium       
rollout       parallel      presenc       20xx          premium       rollout       
upcom         lab           practic       certif        upcom         upcom         
citat         chico         creativ       extens        rollout       citat         
concurr       teach         compress      review        citat         concurr       
control       track         boost         identifi      concurr       control       
overview      case          format        word          control  

In [34]:
topic = np.argsort(document_topics[:, 4])[::-1]
for i in topic[:2]:
        # pshow first two sentences
        print(".".join(processed_user_values[i].split(".")[:4]) + ".\n")

target an internship or entri level opportun in ... softwar engin upcom bsse graduat offer strong foundat in softwar engin and program principl across multipl platform.

my name address phone and email object comput scienc student with profici skill in object orient program comput architectur and the knowledg of mathemat behind comput.commit to achiev project goal in the most effici manner possibl solo or in team.experi name of well respect conveni store ** chang for censorship ** part time clerk extra relief april 2011 current an employe who travel to differ store everi day work shift for employe who have approv time off make posit impress on new manag at variou store everi shift.have work at 70 108 store in the phoenix tucson area highest possibl promot for part time employe 90 th percentil of cash handl in the divis posit feedback from the hundr of manag be met on yearli basi.

